In [ ]:
import pandas
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import numpy as np
from matplotlib import pyplot as plt

shapefile_dir = '../shapefiles/'

print('Packages loaded!')

In [ ]:
health_data_df = pandas.read_excel('neighorhood_health_profiles_all.xlsx', sheet_name='Data', header=0, index_col=0)

tracts_2010_filename = shapefile_dir + 'census_tracts_2010/geo_export_c50bbe56-543e-4878-9c9f-c56be327600a.shp'
tracts_2010_gdf = gpd.read_file(tracts_2010_filename, encoding='utf-8')
tracts_2010_gdf = tracts_2010_gdf.assign(tractname=pandas.to_numeric(tracts_2010_gdf['name'].apply(lambda x: x.split(' ')[-1]), errors='coerce'))
tracts_2010_gdf = tracts_2010_gdf.set_index('tractname')

#tracts_2010_gdf

tract_to_csa_df = pandas.read_csv(shapefile_dir+'census_tract_to_neighborhood.csv')
tract_to_csa_df = tract_to_csa_df.set_index('NAME10')
tracts_2010_gdf = tracts_2010_gdf.join(tract_to_csa_df, how='left')
#tracts_2010_gdf.head(2)

csa_list = tracts_2010_gdf['CSA2010'].drop_duplicates().sort_values().tolist()
csa_df = pandas.DataFrame(data=csa_list, columns=['CSA2010'])
csa_df = csa_df.set_index('CSA2010')
csa_df = csa_df.assign(geometry = np.nan)

for thiscsa, thiscsarow in csa_df.iterrows():
    geolist = []
    for thistract, thistractrow in tracts_2010_gdf.iterrows():
        if (thistractrow['CSA2010'] == thiscsa):
            geolist.append(thistractrow['geometry'])
    csa_df.loc[thiscsa, 'geometry'] = cascaded_union(geolist)

fixtothislist = health_data_df.join(csa_df['geometry'])[health_data_df.join(csa_df['geometry'])['geometry'].isnull()].index.values.tolist()

csa_df = csa_df.reset_index()
csa_df.loc[csa_df['CSA2010'] == 'Allendale/Irvington/S. Hilton', 'CSA2010'] = 'Allendale/Irvington/South Hilton'
csa_df.loc[csa_df['CSA2010'] == 'Glen-Fallstaff', 'CSA2010'] = 'Glen-Falstaff'
csa_df.loc[csa_df['CSA2010'] == 'Mount Washington/Coldspring', 'CSA2010'] = 'Mt. Washington/Coldspring'
csa_df.loc[csa_df['CSA2010'] == 'Westport/Mount Winans/Lakeland', 'CSA2010'] = 'Westport/Mt. Winans/Lakeland'
csa_df = csa_df.set_index('CSA2010')

health_data_df = health_data_df.join(csa_df)

health_data_gdf = gpd.GeoDataFrame(health_data_df, crs=tracts_2010_gdf.crs, geometry='geometry')

water_filename = shapefile_dir + 'water/water.shp'
water_gdf = gpd.read_file(water_filename)
water_gdf = water_gdf.set_index('OBJECTID')
water_gdf = water_gdf.to_crs(tracts_2010_gdf.crs)
print('Read water outlines...')

#health_data_gdf.sample(2).T
health_data_gdf['mortality_diabetes_per_10k'].sample(3).sort_index()

In [ ]:
scalefactor = 0.25 # Use 1 for final map

fig, ax = plt.subplots(figsize=(48*scalefactor,48*scalefactor))

health_data_gdf.plot(column='mortality_diabetes_per_10k', ax=ax, cmap='viridis', linewidth=5*scalefactor)

for ix, thisrow in health_data_gdf[['mortality_diabetes_per_10k', 'geometry']].iterrows():
    x_annot_coord = thisrow.geometry.centroid.x
    y_annot_coord = thisrow.geometry.centroid.y
    if (ix == 'The Waverlies'):
        y_annot_coord = y_annot_coord + scalefactor/500
    elif (ix == 'Fells Point'):
        y_annot_coord = y_annot_coord - scalefactor/500
    else:
        pass
    annotator = ix.replace('-', '-\n')
    annotator = annotator.replace('/', '/\n')
    annotator = annotator + '\n'
    annotator = annotator + str(thisrow['mortality_diabetes_per_10k']) +' per 10k'
    annotator = annotator.strip()
    ax.annotate(annotator, 
                xy=(x_annot_coord, y_annot_coord), 
                xytext=(thisrow.geometry.centroid.x, y_annot_coord), 
                ha='center', va='center', fontsize=24*scalefactor, color='black', weight='normal', backgroundcolor='white')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)
plt.title('Deaths by diabetes in Baltimore community statistical areas', fontsize=72*scalefactor, y = 1 + (0.015 * scalefactor))
water_gdf[water_gdf['NAME'] == 'Harbor'].plot(ax=ax, color='cyan')

# add colorbar
cax = fig.add_axes([0.125, 0.08, 0.775, 0.03])
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=7.5))
# fake up the array of the scalar mappable. Urgh...
sm._A = []

cbar = fig.colorbar(sm, cax=cax, format='%.1f', ticks=np.arange(0, 8, 0.5), orientation='horizontal')

cax.tick_params(labelsize=48*scalefactor)

cbar.set_label('Deaths by diabetes (per 10,000 neighborhood residents)', fontsize=56*scalefactor)

#plt.show()

plt.savefig('maps/diabetes.svg', format='svg')
print('Figure saved!')

In [ ]:
scalefactor = 1#0.25 # Use 1 for final map

fig, ax = plt.subplots(figsize=(48*scalefactor,48*scalefactor))

health_data_gdf.plot(column='median_household_income', ax=ax, cmap='viridis', linewidth=5*scalefactor)

for ix, thisrow in health_data_gdf[['median_household_income', 'geometry']].iterrows():
    x_annot_coord = thisrow.geometry.centroid.x
    y_annot_coord = thisrow.geometry.centroid.y
    if (ix == 'The Waverlies'):
        y_annot_coord = y_annot_coord + scalefactor/500
    elif (ix == 'Fells Point'):
        y_annot_coord = y_annot_coord - scalefactor/500
    else:
        pass
    annotator = ix.replace('-', '-\n')
    annotator = annotator.replace('/', '/\n')
    annotator = annotator + '\n'
    annotator = annotator + '${0:,.0f}'.format(thisrow['median_household_income'])
    annotator = annotator.strip()
    ax.annotate(annotator, 
                xy=(x_annot_coord, y_annot_coord), 
                xytext=(thisrow.geometry.centroid.x, y_annot_coord), 
                ha='center', va='center', fontsize=24*scalefactor, color='black', weight='normal', backgroundcolor='white')

ax.tick_params(axis='both', which='both', bottom=False, left=False, labelleft=False, labelbottom=False)
plt.title('Median household income', fontsize=72*scalefactor, y = 1 + (0.015 * scalefactor))
water_gdf[water_gdf['NAME'] == 'Harbor'].plot(ax=ax, color='cyan')

# add colorbar
cax = fig.add_axes([0.125, 0.08, 0.775, 0.03])
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=110000))
# fake up the array of the scalar mappable. Urgh...
sm._A = []

cbar = fig.colorbar(sm, cax=cax, format='%.0f', ticks=np.arange(0, 140000, 20000), orientation='horizontal')

cax.tick_params(labelsize=48*scalefactor)

cbar.set_label('Median household income ($)', fontsize=56*scalefactor)

#plt.show()
plt.savefig('maps/income.svg', format='svg')
print('Figure saved!')